In [1]:
import datetime
import mysql.connector
import pandas as pd
import os

# Connexion à MySQL
print("Connexion à MySQL...")
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='MichelSardou'  # Remplace par ton mot de passe MySQL
)
cursor = conn.cursor()

print("Création de la base de données...")
cursor.execute("DROP DATABASE IF EXISTS reddit_db")
cursor.execute("CREATE DATABASE reddit_db")
cursor.execute("USE reddit_db")

# Création des tables
print("Création des tables de base...")

cursor.execute("""
    CREATE TABLE AUTHOR (
        author VARCHAR(255) PRIMARY KEY
    )
""")
print("Table AUTHOR créée.")

cursor.execute("""
    CREATE TABLE SUBREDDIT (
        Subreddit_id VARCHAR(255) PRIMARY KEY,
        subreddit VARCHAR(255)
    )
""")
print("Table SUBREDDIT créée.")

cursor.execute("""
    CREATE TABLE SCORE (
        id VARCHAR(255) PRIMARY KEY,
        score INT,
        ups INT,
        downs INT,
        score_hidden BOOLEAN,
        gilded INT
    )
""")
print("Table SCORE créée.")

cursor.execute("""
    CREATE TABLE PARENT (
        parent_id VARCHAR(255) PRIMARY KEY,
        link_id VARCHAR(255)
    )
""")
print("Table PARENT créée.")

cursor.execute("""
    CREATE TABLE REMOVAL (
        removal_reason VARCHAR(255) PRIMARY KEY
    )
""")
print("Table REMOVAL créée.")

cursor.execute("""
    CREATE TABLE CONTROVERSY (
        controversiality INT PRIMARY KEY
    )
""")
print("Table CONTROVERSY créée.")

cursor.execute("""
    CREATE TABLE DISTINGUISHED (
        distinguished VARCHAR(255) PRIMARY KEY
    )
""")
print("Table DISTINGUISHED créée.")

cursor.execute("""
    CREATE TABLE COMMENT (
        Id VARCHAR(255) PRIMARY KEY,
        created_utc DATETIME,
        name VARCHAR(255),
        body TEXT,
        edited BOOLEAN,
        author_flair_css_class VARCHAR(255),
        author_flair_text VARCHAR(255),
        author VARCHAR(255),
        subreddit_id VARCHAR(255),
        controversiality INT,
        FOREIGN KEY (author) REFERENCES AUTHOR(author),
        FOREIGN KEY (subreddit_id) REFERENCES SUBREDDIT(Subreddit_id),
        FOREIGN KEY (controversiality) REFERENCES CONTROVERSY(controversiality)
    )
""")
print("Table COMMENT créée.")

# Tables associatives
print("Création des tables associatives...")

cursor.execute("""
    CREATE TABLE AUTHORED (
        author VARCHAR(255),
        Id VARCHAR(255),
        PRIMARY KEY (author, Id),
        FOREIGN KEY (author) REFERENCES AUTHOR(author),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id)
    )
""")
print("Table AUTHORED créée.")

cursor.execute("""
    CREATE TABLE IS_IN (
        Id VARCHAR(255),
        Subreddit_id VARCHAR(255),
        PRIMARY KEY (Id, Subreddit_id),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (Subreddit_id) REFERENCES SUBREDDIT(Subreddit_id)
    )
""")
print("Table IS_IN créée.")

cursor.execute("""
    CREATE TABLE DEPENDS (
        Id VARCHAR(255),
        parent_id VARCHAR(255),
        PRIMARY KEY (Id, parent_id),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (parent_id) REFERENCES PARENT(parent_id)
    )
""")
print("Table DEPENDS créée.")

cursor.execute("""
    CREATE TABLE HAS_SCORE (
        Id VARCHAR(255),
        score_id VARCHAR(255),
        PRIMARY KEY (Id, score_id),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (score_id) REFERENCES SCORE(id)
    )
""")
print("Table HAS_SCORE créée.")

cursor.execute("""
    CREATE TABLE REMOVED (
        Id VARCHAR(255),
        removal_reason VARCHAR(255),
        PRIMARY KEY (Id, removal_reason),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (removal_reason) REFERENCES REMOVAL(removal_reason)
    )
""")
print("Table REMOVED créée.")

cursor.execute("""
    CREATE TABLE IS_CONTROVERSIAL (
        Id VARCHAR(255),
        controversiality INT,
        PRIMARY KEY (Id, controversiality),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (controversiality) REFERENCES CONTROVERSY(controversiality)
    )
""")
print("Table IS_CONTROVERSIAL créée.")

cursor.execute("""
    CREATE TABLE IS_DISTINGUISHED (
        Id VARCHAR(255),
        distinguished VARCHAR(255),
        PRIMARY KEY (Id, distinguished),
        FOREIGN KEY (Id) REFERENCES COMMENT(Id),
        FOREIGN KEY (distinguished) REFERENCES DISTINGUISHED(distinguished)
    )
""")
print("Table IS_DISTINGUISHED créée.")

conn.commit()
print("Toutes les tables sont créées.\n")

# Chargement des CSV
print("Début de l'importation des CSV...")

folder = "data/excel"
csv_files = {
    'AUTHOR': 'askreddit_author.csv',
    'CONTROVERSY': 'askreddit_controverse.csv',
    'DISTINGUISHED': 'askreddit_distinguihshed.csv',
    'PARENT': 'askreddit_parent.csv',
    'REMOVAL': 'askreddit_removal.csv',
    'SCORE': 'askreddit_score.csv',
    'SUBREDDIT': 'askreddit_subreddit.csv',
    'COMMENT': 'askreddit_comment.csv',
    'DEPENDS': 'askreddit_depends.csv',
    'IS_DISTINGUISHED': 'askreddit_is_distinguihshed.csv',
    'REMOVED': 'askreddit_removed.csv'
}
for table, file in csv_files.items():
    print(f"\nImportation du fichier {file} dans la table {table}...")
    
    df = pd.read_csv(os.path.join(folder, file), dtype=str)
    df = df.where(pd.notnull(df), None)
    
    df.columns = df.columns.map(lambda x: str(x).strip())
    df = df.loc[:, df.columns.notna()]
    df = df.loc[:, df.columns != 'nan']
    
    # Conversion du created_utc
    if 'created_utc' in df.columns:
        df['created_utc'] = df['created_utc'].apply(
            lambda x: datetime.datetime.utcfromtimestamp(int(x)) if x else None
        )
    
    # Conversion de edited
    if 'edited' in df.columns:
        df['edited'] = df['edited'].apply(lambda x: 0 if (x == 'false' or x == '0' or x is None) else 1)
    
    if 'distinguished' in df.columns:
        df = df.filter(df['distinguished'].notnull() & (df['distinguished'] != ''))
    
    clean_cols = list(df.columns)
    print(f"Colonnes utilisées pour {table} : {clean_cols}")

    cols = ",".join(clean_cols)
    placeholders = ",".join(["%s"] * len(clean_cols))
    insert_stmt = f"INSERT INTO {table} ({cols}) VALUES ({placeholders})"

    for row in df[clean_cols].itertuples(index=False):
        cursor.execute(insert_stmt, tuple(row))
    
    print(f"{len(df)} lignes insérées dans {table}.")

cursor.close()
conn.close()
print("Connexion fermée.")

Connexion à MySQL...
Création de la base de données...
Création des tables de base...
Table AUTHOR créée.
Table SUBREDDIT créée.
Table SCORE créée.
Table PARENT créée.
Table REMOVAL créée.
Table CONTROVERSY créée.
Table DISTINGUISHED créée.
Table COMMENT créée.
Création des tables associatives...
Table AUTHORED créée.
Table IS_IN créée.
Table DEPENDS créée.
Table HAS_SCORE créée.
Table REMOVED créée.
Table IS_CONTROVERSIAL créée.
Table IS_DISTINGUISHED créée.
Toutes les tables sont créées.

Début de l'importation des CSV...

Importation du fichier askreddit_author.csv dans la table AUTHOR...
Colonnes utilisées pour AUTHOR : ['author']
570735 lignes insérées dans AUTHOR.

Importation du fichier askreddit_controverse.csv dans la table CONTROVERSY...
Colonnes utilisées pour CONTROVERSY : ['controversiality']
2 lignes insérées dans CONTROVERSY.

Importation du fichier askreddit_distinguihshed.csv dans la table DISTINGUISHED...
Colonnes utilisées pour DISTINGUISHED : []
2 lignes insérées da